## Particle Simulator in Cython

#### This program will simulate particles moving in a circular path at a constant speed. 
#### The purpose of this simulation is to improve performance of python program.

In [1]:
%load_ext cython
# loading cython extension

In [2]:
%%cython

# Importing required libraries
import numpy as np
from random import uniform
from libc.math cimport sqrt

# This function is written in Cython and used to simulate particles
def c_evolve(double[:, :] r_i, double[:] ang_speed_i, int timestep, int nsteps):
    cdef double norm, x, y, vx, vy, dx, dy, ang_speed
    
    v_i = np.empty_like(r_i)
    
    cdef int i, j
    cdef int nparticles = r_i.shape[0]
    
    for i in range(nsteps):
        for j in range(nparticles):
            x = r_i[j, 0]
            y = r_i[j, 1]
            
            ang_speed = ang_speed_i[j]
            
            norm = sqrt(x**2 + y**2)
            
            vx = (-y)/norm
            vy = x/norm
            
            dx = timestep * ang_speed * vx
            dy = timestep * ang_speed * vy
            
            r_i[j, 0] += dx
            r_i[j, 1] += dy
            
            
# The particles class has three variables x, y positions and angular velocity            
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel


# To simulate the particles
class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    # The first method utilizes numpy for simulation
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt/timestep)
        # nsteps = 1000 if dt = 0.01

        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
        
        for i in range(nsteps):
            
            norm_ii = (r_i ** 2).sum(axis=1)
            norm_i = np.sqrt(norm_ii)
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] *= -1
            v_i /= norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
            
            for i, p in enumerate(self.particles):
                p.x, p.y = r_i[i]

    # The second method utilizes cython for simulation        
    def evolve_cython(self, dt):
        timestep = 0.00001
        nsteps = int(dt/timestep)
        # nsteps = 1000 if dt = 0.01

        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_speed_i = np.array([p.ang_vel for p in self.particles])
        
        c_evolve(r_i, ang_speed_i, timestep, nsteps)
        
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
            
            
# To benchmark the different methods
def benchmark(npart=100, method='cython'):
    particles = [
        Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0))
        for i in range(npart)
    ]

    simulator = ParticleSimulator(particles)
    
    if method == 'cython':
        simulator.evolve_cython(0.1)
        
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

### The result of Cython

In [3]:
%timeit benchmark(100, "cython")

99.5 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### The result of Numpy

In [4]:
%timeit benchmark(100, 'numpy')

1.25 s ± 45.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
